In [1]:
import os  
import torch
import numpy as np
import os.path

In [2]:
def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')

 
def load_data(split_load, actions_dict, GT_folder, DATA_folder):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
  
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']

    data_breakfast = []
    labels_breakfast = []
    tasks_breakfast = []
    for content in content_all:
        curr_task = content.split('_')[-1].split('.')[0]
        tasks_breakfast.append(int( all_tasks.index(curr_task)) )

        file_ptr = open( GT_folder + content, 'r')
        curr_gt = file_ptr.read().split('\n')[:-1]
        label_seq, length_seq = get_label_length_seq(curr_gt)

        loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        curr_data = np.loadtxt(loc_curr_data, dtype='double')

        label_curr_video = []
        for iik in range(len(curr_gt)):
            label_curr_video.append( actions_dict[curr_gt[iik]] )
  
        data_breakfast.append(torch.tensor(curr_data,  dtype=torch.double ) )
        labels_breakfast.append(label_curr_video )
 
    return   data_breakfast, labels_breakfast, tasks_breakfast


def get_label_bounds( data_labels):
    labels_uniq = []
    labels_uniq_loc = []
    for kki in range(0, len(data_labels) ):
        uniq_group, indc_group = get_label_length_seq(data_labels[kki])
        labels_uniq.append(uniq_group)
        labels_uniq_loc.append(indc_group)
    return labels_uniq, labels_uniq_loc

def get_label_length_seq(content):
    label_seq = []
    length_seq = []
    start = 0
    length_seq.append(0)
    for i in range(len(content)):
        if content[i] != content[start]:
            label_seq.append(content[start])
            length_seq.append(i)
            start = i
    label_seq.append(content[start])
    length_seq.append(len(content))

    return label_seq, length_seq


def get_maxpool_lstm_data(cData, indices):
    list_data = []
    for kkl in range(len(indices)-1):
        cur_start = indices[kkl]
        cur_end = indices[kkl+1]
        if cur_end > cur_start:
            list_data.append(torch.max(cData[cur_start:cur_end,:],
                                       0)[0].squeeze(0))
        else:
            list_data.append(torch.max(cData[cur_start:cur_end+1,:],
                                       0)[0].squeeze(0))
    list_data  =  torch.stack(list_data)
    return list_data

def read_mapping_dict(mapping_file):
    file_ptr = open(mapping_file, 'r')
    actions = file_ptr.read().split('\n')[:-1]

    actions_dict=dict()
    for a in actions:
        actions_dict[a.split()[1]] = int(a.split()[0])

    return actions_dict


#split = 'test'
COMP_PATH = ''
    
train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle')
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle')
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/')
DATA_folder =  os.path.join(COMP_PATH, 'data/')
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt')

  
actions_dict = read_mapping_dict(mapping_loc)

data_feat_train, data_labels_train, tasks_labels_train = load_data(train_split, actions_dict, GT_folder, DATA_folder)

data_feat_test, data_labels_test, tasks_labels_test= load_data( test_split, actions_dict, GT_folder, DATA_folder)
print('total number videos ' +  str(len(data_labels_train))  )

total number videos 1460


In [3]:
data_feat_train

[tensor([[-13.5283,   5.3792,  -0.2658,  ...,  -2.9244,  -3.2108,  -9.9336],
         [-11.9813,   5.8757,  -2.2880,  ...,  -2.4845,  -2.9390, -11.4061],
         [-13.2690,   3.9517,  -2.9547,  ...,  -1.6744,   0.0268, -10.6195],
         ...,
         [ -8.7294,  10.9188,   3.2959,  ...,  -3.8530,   1.0943,   1.5798],
         [ -7.3483,   7.9912,   2.0413,  ...,  -3.6108,  -0.1137,   2.2453],
         [ -6.2605,   7.8383,   2.1280,  ...,  -2.0596,   1.5379,   0.5398]],
        dtype=torch.float64),
 tensor([[-9.5425,  6.7161, -2.7519,  ..., -1.7725,  2.9804, -0.1084],
         [-9.2990,  4.6064, -3.7513,  ..., -3.5214,  1.1276,  0.5704],
         [-8.8719,  5.7336, -0.8734,  ..., -2.7662,  0.8786, -1.9142],
         ...,
         [-3.7001,  3.5948,  5.9831,  ..., -2.8416, 15.2684,  1.2916],
         [-2.4362,  4.2436,  4.6704,  ..., -4.5087, 15.4034,  1.5560],
         [-2.6052,  3.3519,  5.5914,  ..., -3.7740, 14.4137,  1.4882]],
        dtype=torch.float64),
 tensor([[-10.9121,  1

In [4]:
data_labels_train[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [5]:
tasks_labels_train

[1,
 3,
 4,
 5,
 8,
 9,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 9,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 6,
 7,
 0,
 1,
 2,
 5,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 0,
 1,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 9,
 6,
 7,
 0,
 1,
 2,
 5,
 0,
 1,
 2,
 4,
 5,
 8,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 5,
 0,
 2,
 3,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 7,
 0,
 1,
 2,
 3,
 5,
 0,
 1,
 2,
 4,
 5,
 8,
 9,
 6,
 7,
 0,
 1,
 2,
 3,
 4,


In [6]:
len(data_labels_train[2])

2238

In [7]:
len(data_feat_train)

1460

In [8]:
frames=[]
train_labels=[]
for i in range(len(data_labels_train)):
    for j in range(len(data_labels_train[i])):
        if j==0:
            frame=[data_feat_train[i][j]]
            train_labels.append(data_labels_train[i][j])
        elif data_labels_train[i][j]==data_labels_train[i][j-1]:
            frame.append(data_feat_train[i][j])
        else:
            frames.append(torch.stack(frame))
            train_labels.append(data_labels_train[i][j])
            frame=[data_feat_train[i][j]]
    frames.append(torch.stack(frame))

In [9]:
print(len(train_labels))

9885


In [10]:
print(len(frames))

9885


In [11]:
seq_len=[]
for i in range(len(frames)):
    seq_len.append(len(frames[i]))

In [12]:
seq_len

[9,
 260,
 205,
 70,
 48,
 328,
 261,
 633,
 4236,
 1293,
 72,
 61,
 6,
 50,
 162,
 365,
 118,
 296,
 109,
 522,
 133,
 306,
 99,
 72,
 23,
 290,
 165,
 140,
 125,
 141,
 15,
 245,
 180,
 274,
 562,
 64,
 526,
 201,
 5408,
 76,
 379,
 110,
 64,
 1230,
 90,
 370,
 105,
 1210,
 730,
 230,
 107,
 21,
 405,
 630,
 310,
 290,
 90,
 191,
 3,
 39,
 220,
 100,
 9,
 260,
 205,
 71,
 49,
 240,
 99,
 183,
 48,
 328,
 261,
 633,
 4236,
 1293,
 72,
 61,
 6,
 50,
 162,
 365,
 118,
 296,
 109,
 522,
 133,
 306,
 99,
 72,
 12,
 301,
 165,
 140,
 125,
 141,
 3,
 257,
 180,
 274,
 562,
 64,
 526,
 201,
 5408,
 76,
 379,
 110,
 64,
 1230,
 90,
 370,
 105,
 1210,
 730,
 230,
 107,
 21,
 405,
 630,
 310,
 290,
 90,
 191,
 140,
 820,
 195,
 360,
 118,
 324,
 298,
 378,
 2102,
 152,
 241,
 63,
 3,
 39,
 220,
 100,
 9,
 260,
 205,
 70,
 48,
 328,
 261,
 633,
 4236,
 1293,
 72,
 61,
 6,
 50,
 162,
 365,
 118,
 296,
 109,
 522,
 133,
 306,
 99,
 72,
 27,
 286,
 165,
 140,
 125,
 141,
 65,
 195,
 180,
 274,
 562

In [13]:
import matplotlib.pyplot as plt
plt.hist(seq_len,bins=20)

(array([6.974e+03, 1.678e+03, 5.560e+02, 2.070e+02, 1.560e+02, 8.100e+01,
        5.500e+01, 4.500e+01, 1.700e+01, 3.300e+01, 2.600e+01, 1.700e+01,
        0.000e+00, 6.000e+00, 1.300e+01, 1.200e+01, 0.000e+00, 0.000e+00,
        7.000e+00, 2.000e+00]),
 array([2.00000e+00, 2.91450e+02, 5.80900e+02, 8.70350e+02, 1.15980e+03,
        1.44925e+03, 1.73870e+03, 2.02815e+03, 2.31760e+03, 2.60705e+03,
        2.89650e+03, 3.18595e+03, 3.47540e+03, 3.76485e+03, 4.05430e+03,
        4.34375e+03, 4.63320e+03, 4.92265e+03, 5.21210e+03, 5.50155e+03,
        5.79100e+03]),
 <a list of 20 Patch objects>)

In [14]:
seq=[]
seq_label=[]
for i in range(len(frames)):
    seq.append(frames[i][:512])
    seq_label.append(train_labels[i])

In [16]:
def pad(s,length):
    while(len(s)<length):
        s=torch.cat((s,torch.zeros(1,400)))
    return s

In [20]:
from torch.utils.data import Dataset
class SeqDataset(Dataset):
    def __init__(self):
        super(SeqDataset, self).__init__()
        Seq = seq
        Seq_label = seq_label
        self.data=[]
        self.pad_data=[]
        for i in range(len(Seq)):
            self.data.append([Seq[i],Seq_label[i]])
    def __getitem__(self, index): 
        return [pad(self.data[index][0],512),self.data[index][1]]

    def __len__(self):
        return len(self.data)

In [21]:
data=SeqDataset()

In [22]:
frames_test=[]
test_labels=[]
for i in range(len(data_labels_test)):
    for j in range(len(data_labels_test[i])):
        if j==0:
            frame_test=[data_feat_test[i][j]]
            test_labels.append(data_labels_test[i][j])
        elif data_labels_test[i][j]==data_labels_test[i][j-1]:
            frame_test.append(data_feat_test[i][j])
        else:
            frames_test.append(torch.stack(frame_test))
            test_labels.append(data_labels_test[i][j])
            frame_test=[data_feat_test[i][j]]
    frames_test.append(torch.stack(frame))

In [23]:
seqtest=[]
seqtest_label=[]
for i in range(len(frames_test)):
    seqtest.append(frames_test[i][:512])
    seqtest_label.append(test_labels[i])

In [24]:
class SeqTestDataset(Dataset):
    def __init__(self):
        super(SeqTestDataset, self).__init__()
        Seq = seqtest
        Seq_label = seqtest_label
        self.data=[]
        self.pad_data=[]
        for i in range(len(Seq)):
            self.data.append([Seq[i],Seq_label[i]])
    def __getitem__(self, index): 
        return [pad(self.data[index][0],512),self.data[index][1]]

    def __len__(self):
        return len(self.data)

In [25]:
datatest=SeqTestDataset()

In [26]:
from torch.utils.data import DataLoader
Data_train= DataLoader(data, shuffle=True, batch_size=64)
Data_test= DataLoader(datatest, batch_size=64)

In [27]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size=400, hidden_layer_size=400, output_size=48):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = torch.nn.LSTM(input_size, hidden_layer_size, bidirectional= True, num_layers=2 ,batch_first=True)
        self.pool = torch.nn.MaxPool2d((512,1))
        self.linear = torch.nn.Linear(2*hidden_layer_size,hidden_layer_size)
        self.linear2 = torch.nn.Linear(hidden_layer_size, output_size)
        self.relu= torch.nn.ReLU()
        self.hidden = torch.zeros(4,64,self.hidden_layer_size)
        self.cell = torch.zeros(4,64,self.hidden_layer_size)

    def forward(self, input_seq):
        lstm_out, (self.hidden,self.cell) = self.lstm(input_seq.view(len(input_seq) ,input_seq.shape[1], -1), (self.hidden,self.cell))
        l1 = self.linear(lstm_out.view(len(input_seq),input_seq.shape[1],-1))
        r = self.relu(l1.view(len(input_seq),input_seq.shape[1],-1))
        pooled_out=self.pool(r)
        predictions = self.linear2(pooled_out.view(len(input_seq), -1))
        return predictions

In [28]:
model = LSTM()
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [29]:
print(model)

LSTM(
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, bidirectional=True)
  (pool): MaxPool2d(kernel_size=(512, 1), stride=(512, 1), padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=800, out_features=400, bias=True)
  (linear2): Linear(in_features=400, out_features=48, bias=True)
  (relu): ReLU()
)


In [30]:
#print(data_32[0][0])

In [31]:
epochs = 15
for i in range(epochs):
    acc=[]
    for seq, labels in Data_train:
        optimizer.zero_grad()
        model.hidden = torch.zeros(4, len(seq), model.hidden_layer_size)
        model.cell = torch.zeros(4, len(seq), model.hidden_layer_size)
        y_pred = model(seq.float())
        single_loss = loss_function(y_pred, labels)
        preds = torch.argmax(y_pred, dim=1).flatten()
        accuracy = (preds == labels).numpy().mean()
        acc.append(accuracy)
        single_loss.backward()
        optimizer.step()

    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
    print(f'epoch: {i:3} accuracy: {np.mean(acc):10.8f}')
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   0 loss: 1.70502687
epoch:   0 accuracy: 0.38537264
epoch:   1 loss: 2.08527613
epoch:   1 accuracy: 0.48853587
epoch:   2 loss: 1.47561574
epoch:   2 accuracy: 0.56037264
epoch:   3 loss: 1.41927981
epoch:   3 accuracy: 0.61666435
epoch:   4 loss: 1.02550626
epoch:   4 accuracy: 0.64846009


KeyboardInterrupt: 

In [ ]:
model.eval()
val_acc=[]
for seq, labels in Data_test:
    with torch.no_grad():
        model.hidden = torch.zeros(4, len(seq), model.hidden_layer_size)
        model.cell = torch.zeros(4, len(seq), model.hidden_layer_size)
        y_pred = model(seq.float())
    val_loss = loss_function(y_pred, labels)
    preds = torch.argmax(y_pred, dim=1).flatten()
    val_accuracy = (preds == labels).numpy().mean()
    val_acc.append(accuracy)


print(f'Vaidation accuracy: {np.mean(val_acc):10.8f}')